In [1]:
import threading
import cv2
from segmentation_models import Unet
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint,TensorBoard
from sklearn.model_selection import train_test_split
import os
from keras import optimizers
from os.path import join
import h5py
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from albumentations import (
    CLAHE, RandomRotate90, Transpose, ShiftScaleRotate, Blur, OpticalDistortion, 
    GridDistortion, HueSaturationValue, IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, 
    MedianBlur, IAAPiecewiseAffine, IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, 
    Flip, OneOf, Compose,HorizontalFlip
)
from losses_original import (
    binary_crossentropy,
    dice_loss,
    bce_dice_loss,
    dice_coef,
    weighted_bce_dice_loss,my_iou_metric,
    my_iou_metric_2,IOU_Metric_new,my_iou_metric_bowl,
    competition_metric,
    iou_bce_loss,lovasz_loss,competition_metric_loss
)


os.environ["CUDA_VISIBLE_DEVICES"]="0"
WIDTH = 128
HEIGHT = 128
BATCH_SIZE = 32
number_folds = 4
weg_train_img = '/mnt/ssd1/dataset/salt/images-train/'
weg_train_mask = '/mnt/ssd1/dataset/salt/fixed_train_mask/'
weg_train_mask = '/mnt/ssd1/dataset/salt/masks-train/'

Using TensorFlow backend.


In [2]:
#5
def strong_aug(p=0.5):
    return Compose([
        HorizontalFlip(),
        RandomBrightness(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0., rotate_limit=5, p=0.75),
    ], p=p)
augmentation = strong_aug(p=0.5)


In [3]:
def clean(value):
    return value.replace('.png','')

df = pd.read_csv('/mnt/ssd1/dataset/salt/train.csv')
df['folds'] = range(len(df))
df.folds = df.folds % number_folds

train_ids = df.id.values
train_folds = df.folds.values
df['ids_train_batch'] = train_ids
df['fold'] = train_folds


In [4]:
def custom_pad(img, pad_size=256, mode=3):
    x1 = (pad_size - img.shape[0]) // 2
    x2 = pad_size - img.shape[0] - x1

    if mode == 0:
        img = cv2.copyMakeBorder(img, x1, x2, x1, x2, cv2.BORDER_CONSTANT)

    if mode == 1:
        img = cv2.copyMakeBorder(img, x1, x2, x1, x2, cv2.BORDER_REPLICATE)

    if mode == 2:
        img = cv2.copyMakeBorder(img, x1, x2, x1, x2, cv2.BORDER_REFLECT_101)

    if mode == 3:
        img = cv2.copyMakeBorder(img, x1, x2, 0, 0, cv2.BORDER_REPLICATE)
        img = cv2.copyMakeBorder(img, 0, 0, x1, x2, cv2.BORDER_REFLECT_101)

    if mode == 4:
        img = cv2.copyMakeBorder(img, x1, x2, 0, 0, cv2.BORDER_REFLECT_101)
        img = cv2.copyMakeBorder(img, 0, 0, x1, x2, cv2.BORDER_REPLICATE)

    return img

In [5]:
WIDTH = 128
HEIGHT = 128
BATCH_SIZE = 32

pad = 77
load_img = lambda im: np.pad(cv2.imread(join(weg_train_img, '{}.png'.format(im))) #[:,:,0:1]\
, ((pad,pad+1),(pad,pad+1),(0,0)), 'constant')
load_mask = lambda im: np.pad(cv2.imread(join(weg_train_mask, '{}.png'.format(im)))[:,:,0]\
                              , ((pad,pad+1),(pad,pad+1)), 'constant')

load_img = lambda im: custom_pad(cv2.imread(join(weg_train_img, '{}.png'.format(im))))
load_mask = lambda im: custom_pad(cv2.imread(join(weg_train_mask, '{}.png'.format(im)))[:,:,0])\


class ThreadSafeIterator:

    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()


def threadsafe_generator(f):
    """
    A decorator that takes a generator function and makes it thread-safe.
    """

    def g(*args, **kwargs):
        return ThreadSafeIterator(f(*args, **kwargs))

    return g


@threadsafe_generator
def train_generator(df):
    while True:
        shuffle_indices = np.arange(len(df))
        shuffle_indices = np.random.permutation(shuffle_indices)
        
        for start in range(0, len(df), BATCH_SIZE):
            x_batch = []
            y_batch = []
            
            end = min(start + BATCH_SIZE, len(df))
            ids_train_batch = df.iloc[shuffle_indices[start:end]]
            
            for _id in ids_train_batch.values:
                img = load_img(_id)/ 255.
                
                mask = load_mask(_id)
                mask = np.expand_dims(mask, axis=-1)
                mask = mask/255.
                assert mask.ndim == 3
                
                # === You can add data augmentations here. === #
                #if np.random.random() < 0.5:
                 #   img, mask = img[:, ::-1, :], mask[..., ::-1, :]  # random horizontal flip
                data = {"image": img, "mask": mask}
                augmented = augmentation(**data)
                img, mask = augmented["image"], augmented["mask"]

                if len(img.shape) < 3:
                    img = np.expand_dims(img, axis=-1)
                if len(mask.shape) < 3:
                    mask = np.expand_dims(mask, axis=-1)
                    
                x_batch.append(img)
                y_batch.append(mask)
            
            x_batch = np.array(x_batch, np.float32) 
            y_batch = np.array(y_batch, np.float32) 
            

            
            yield x_batch, y_batch


@threadsafe_generator
def valid_generator(df):
    while True:
        for start in range(0, len(df), BATCH_SIZE):
            x_batch = []
            y_batch = []

            end = min(start + BATCH_SIZE, len(df))
            ids_train_batch = df.iloc[start:end]

            for _id in ids_train_batch.values:
                img = load_img(_id)
                mask = load_mask(_id)


                mask = np.expand_dims(mask, axis=-1)
                assert mask.ndim == 3
                
                x_batch.append(img)
                y_batch.append(mask)

            x_batch = np.array(x_batch, np.float32) / 255.
            y_batch = np.array(y_batch, np.float32) / 255.

            yield x_batch, y_batch

In [24]:
epochs = 20000
BATCH_SIZE = 8
!rm -r logs

input_1 = 128
input_2 = 128
for fold_number in [0]:
    print ( 50*'-', ' FOLD ' + str(fold_number), 50*'-' )
    ids_train = df.ids_train_batch[df.fold != fold_number]
    ids_valid = df.ids_train_batch[df.fold == fold_number]

    model = Unet(backbone_name='resnet34', encoder_weights='imagenet',input_shape=(256,256,3),freeze_encoder=True,
                            decoder_use_batchnorm = True,decoder_block_type = 'upsampling')
    model.compile(optimizers.Adadelta(), competition_metric_loss, ['accuracy',competition_metric])



    callbacks = [EarlyStopping(monitor='val_competition_metric',
                               patience=5,
                               verbose=1,
                               min_delta=1e-12,
                               mode='max'),
                 ReduceLROnPlateau(monitor='val_competition_metric',
                                   factor=0.2,
                                   patience=2,
                                   verbose=1,
                                   epsilon=1e-12,
                                   mode='max'),
                 ModelCheckpoint(monitor='val_competition_metric',
                                 filepath='1_origina_model_weights_fold_{}.hdf5'.format(fold_number),
                                 save_best_only=True,verbose = 1,
                                 mode='max')]
    print('number of train samples in fold ' +str(fold_number) + ' is ' + str(len(ids_train)))
    print('number of validation samples in fold ' +str(fold_number) + ' is ' + str(len(ids_valid)))
    
    #model.load_weights('AUTOENCODER.hdf5'.format(fold_number))
    model.fit_generator(generator=train_generator(ids_train),
                        steps_per_epoch=np.ceil(float(len(ids_train)) / float(BATCH_SIZE)),
                        epochs=10000,
                        verbose=1,
                        callbacks=callbacks,
                        validation_data=valid_generator(ids_valid),
                        validation_steps=np.ceil(float(len(ids_valid)) / float(BATCH_SIZE)))
    
    del model



rm: cannot remove 'logs': No such file or directory
--------------------------------------------------  FOLD 0 --------------------------------------------------


/home/n01z3/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


number of train samples in fold 0 is 3000
number of validation samples in fold 0 is 1000
Epoch 1/10000
375/375 [==============================] - 93s 247ms/step - loss: 0.9975 - acc: 0.6362 - competition_metric: 0.3820 - val_loss: 1.5413 - val_acc: 0.6500 - val_competition_metric: 0.3890

Epoch 00001: val_competition_metric improved from -inf to 0.38900, saving model to 1_origina_model_weights_fold_0.hdf5
Epoch 2/10000
375/375 [==============================] - 58s 155ms/step - loss: 0.8288 - acc: 0.7340 - competition_metric: 0.4582 - val_loss: 1.2900 - val_acc: 0.6122 - val_competition_metric: 0.3350

Epoch 00002: val_competition_metric did not improve from 0.38900
Epoch 3/10000
375/375 [==============================] - 61s 163ms/step - loss: 0.7408 - acc: 0.7549 - competition_metric: 0.5088 - val_loss: 1.3198 - val_acc: 0.5171 - val_competition_metric: 0.2875

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.2.

Epoch 00003: val_competition_metric did not improve from 0.38

In [25]:
# epochs = 20000
BATCH_SIZE = 16
!rm -r logs

input_1 = 128
input_2 = 128
for fold_number in [0,1]:
    print ( 50*'-', ' FOLD ' + str(fold_number), 50*'-' )
    ids_train = df.ids_train_batch[df.fold != fold_number]
    ids_valid = df.ids_train_batch[df.fold == fold_number]

    model = Unet(backbone_name='resnet34', encoder_weights='imagenet',input_shape=(256,256,3),freeze_encoder=False,
                            decoder_use_batchnorm = True,decoder_block_type = 'upsampling')
    model.compile(optimizers.Adam(lr = 0.0001), bce_dice_loss, ['accuracy',competition_metric])

    callbacks = [EarlyStopping(monitor='val_competition_metric',
                               patience=20,
                               verbose=1,
                               min_delta=1e-12,
                               mode='max'),
                 ReduceLROnPlateau(monitor='val_competition_metric',
                                   factor=0.2,
                                   patience=4,
                                   verbose=1,
                                   epsilon=1e-12,
                                   mode='max'),
                 ModelCheckpoint(monitor='val_competition_metric',
                                 filepath='2_origina_model_weights_fold_{}.hdf5'.format(fold_number),
                                 save_best_only=True,verbose = 1,
                                 mode='max')]
    print('number of train samples in fold ' +str(fold_number) + ' is ' + str(len(ids_train)))
    print('number of validation samples in fold ' +str(fold_number) + ' is ' + str(len(ids_valid)))
    
    model.load_weights('1_origina_model_weights_fold_{}.hdf5'.format(fold_number))
    #model.load_weights('AUTOENCODER.hdf5'.format(fold_number))
    model.fit_generator(generator=train_generator(ids_train),
                        steps_per_epoch=np.ceil(float(len(ids_train)) / float(BATCH_SIZE)),
                        epochs=10000,
                        verbose=1,
                        callbacks=callbacks,
                        validation_data=valid_generator(ids_valid),
                        validation_steps=np.ceil(float(len(ids_valid)) / float(BATCH_SIZE)))
    
    del model



rm: cannot remove 'logs': No such file or directory
--------------------------------------------------  FOLD 0 --------------------------------------------------


/home/n01z3/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


number of train samples in fold 0 is 3000
number of validation samples in fold 0 is 1000
Epoch 1/10000
188/188 [==============================] - 120s 638ms/step - loss: 0.3343 - acc: 0.7047 - competition_metric: 0.5237 - val_loss: 0.3659 - val_acc: 0.6672 - val_competition_metric: 0.5606

Epoch 00001: val_competition_metric improved from -inf to 0.56060, saving model to 2_origina_model_weights_fold_0.hdf5
Epoch 2/10000
188/188 [==============================] - 72s 385ms/step - loss: 0.2551 - acc: 0.7189 - competition_metric: 0.6344 - val_loss: 0.2594 - val_acc: 0.7335 - val_competition_metric: 0.6472

Epoch 00002: val_competition_metric improved from 0.56060 to 0.64720, saving model to 2_origina_model_weights_fold_0.hdf5
Epoch 3/10000
188/188 [==============================] - 74s 394ms/step - loss: 0.2142 - acc: 0.7331 - competition_metric: 0.6884 - val_loss: 0.2335 - val_acc: 0.7554 - val_competition_metric: 0.6982

Epoch 00003: val_competition_metric improved from 0.64720 to 0.698

188/188 [==============================] - 74s 396ms/step - loss: 0.0394 - acc: 0.6801 - competition_metric: 0.8881 - val_loss: 0.2363 - val_acc: 0.6599 - val_competition_metric: 0.7743

Epoch 00029: val_competition_metric did not improve from 0.78560
Epoch 30/10000
188/188 [==============================] - 74s 394ms/step - loss: 0.0385 - acc: 0.6805 - competition_metric: 0.8830 - val_loss: 0.2378 - val_acc: 0.6612 - val_competition_metric: 0.7730

Epoch 00030: val_competition_metric did not improve from 0.78560
Epoch 31/10000
188/188 [==============================] - 74s 395ms/step - loss: 0.0389 - acc: 0.6812 - competition_metric: 0.8800 - val_loss: 0.2373 - val_acc: 0.6575 - val_competition_metric: 0.7741

Epoch 00031: val_competition_metric did not improve from 0.78560
Epoch 32/10000
188/188 [==============================] - 74s 394ms/step - loss: 0.0389 - acc: 0.6808 - competition_metric: 0.8829 - val_loss: 0.2390 - val_acc: 0.6621 - val_competition_metric: 0.7725

Epoch 00032:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



188/188 [==============================] - 74s 396ms/step - loss: 0.0420 - acc: 0.6984 - competition_metric: 0.8782 - val_loss: 0.2254 - val_acc: 0.6739 - val_competition_metric: 0.7869

Epoch 00025: val_competition_metric did not improve from 0.79130
Epoch 26/10000
188/188 [==============================] - 74s 396ms/step - loss: 0.0405 - acc: 0.6985 - competition_metric: 0.8827 - val_loss: 0.2264 - val_acc: 0.6737 - val_competition_metric: 0.7860

Epoch 00026: val_competition_metric did not improve from 0.79130
Epoch 27/10000
188/188 [==============================] - 75s 396ms/step - loss: 0.0403 - acc: 0.6891 - competition_metric: 0.8854 - val_loss: 0.2346 - val_acc: 0.6671 - val_competition_metric: 0.7797

Epoch 00027: ReduceLROnPlateau reducing learning rate to 3.999999898951501e-06.

Epoch 00027: val_competition_metric did not improve from 0.79130
Epoch 28/10000
188/188 [==============================] - 74s 395ms/step - loss: 0.0387 - acc: 0.6839 - competition_metric: 0.8878 - 

In [26]:
epochs = 20000
BATCH_SIZE = 16

input_1 = 128
input_2 = 128
for fold_number in [0,1]:
    print ( 50*'-', ' FOLD ' + str(fold_number), 50*'-' )
    ids_train = df.ids_train_batch[df.fold != fold_number]
    ids_valid = df.ids_train_batch[df.fold == fold_number]

    model = Unet(backbone_name='resnet34', encoder_weights=None,input_shape=(256,256,3),freeze_encoder=False,
                            decoder_use_batchnorm = True,decoder_block_type = 'upsampling')
    model.compile(optimizers.Adam(lr = 0.0001), bce_dice_loss, ['accuracy',competition_metric])

    callbacks = [EarlyStopping(monitor='val_competition_metric',
                               patience=20,
                               verbose=1,
                               min_delta=1e-12,
                               mode='max'),
                 ReduceLROnPlateau(monitor='val_competition_metric',
                                   factor=0.2,
                                   patience=4,
                                   verbose=1,
                                   epsilon=1e-12,
                                   mode='max'),
                 ModelCheckpoint(monitor='val_competition_metric',
                                 filepath='3_origina_model_weights_fold_{}.hdf5'.format(fold_number),
                                 save_best_only=True,verbose = 1,
                                 mode='max')]
    print('number of train samples in fold ' +str(fold_number) + ' is ' + str(len(ids_train)))
    print('number of validation samples in fold ' +str(fold_number) + ' is ' + str(len(ids_valid)))
    
    model.load_weights('1_origina_model_weights_fold_{}.hdf5'.format(fold_number))
    #model.load_weights('AUTOENCODER.hdf5'.format(fold_number))
    model.fit_generator(generator=train_generator(ids_train),
                        steps_per_epoch=np.ceil(float(len(ids_train)) / float(BATCH_SIZE)),
                        epochs=10000,
                        verbose=1,
                        callbacks=callbacks,
                        validation_data=valid_generator(ids_valid),
                        validation_steps=np.ceil(float(len(ids_valid)) / float(BATCH_SIZE)))
    
    del model



--------------------------------------------------  FOLD 0 --------------------------------------------------


/home/n01z3/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


number of train samples in fold 0 is 3000
number of validation samples in fold 0 is 1000
Epoch 1/10000
188/188 [==============================] - 125s 665ms/step - loss: 0.3368 - acc: 0.7126 - competition_metric: 0.5291 - val_loss: 0.2955 - val_acc: 0.7118 - val_competition_metric: 0.6178

Epoch 00001: val_competition_metric improved from -inf to 0.61780, saving model to 3_origina_model_weights_fold_0.hdf5
Epoch 2/10000
188/188 [==============================] - 76s 406ms/step - loss: 0.2533 - acc: 0.7228 - competition_metric: 0.6376 - val_loss: 0.2570 - val_acc: 0.7384 - val_competition_metric: 0.6457

Epoch 00002: val_competition_metric improved from 0.61780 to 0.64570, saving model to 3_origina_model_weights_fold_0.hdf5
Epoch 3/10000
188/188 [==============================] - 76s 405ms/step - loss: 0.2073 - acc: 0.7167 - competition_metric: 0.6823 - val_loss: 0.2323 - val_acc: 0.7289 - val_competition_metric: 0.6770

Epoch 00003: val_competition_metric improved from 0.64570 to 0.677

188/188 [==============================] - 74s 395ms/step - loss: 0.0427 - acc: 0.6856 - competition_metric: 0.8757 - val_loss: 0.2331 - val_acc: 0.6727 - val_competition_metric: 0.7817

Epoch 00028: val_competition_metric did not improve from 0.78310
Epoch 29/10000
188/188 [==============================] - 74s 396ms/step - loss: 0.0415 - acc: 0.6835 - competition_metric: 0.8789 - val_loss: 0.2349 - val_acc: 0.6694 - val_competition_metric: 0.7817

Epoch 00029: val_competition_metric did not improve from 0.78310
Epoch 30/10000
188/188 [==============================] - 74s 394ms/step - loss: 0.0405 - acc: 0.6805 - competition_metric: 0.8807 - val_loss: 0.2366 - val_acc: 0.6645 - val_competition_metric: 0.7805

Epoch 00030: val_competition_metric did not improve from 0.78310
Epoch 31/10000
188/188 [==============================] - 74s 395ms/step - loss: 0.0403 - acc: 0.6776 - competition_metric: 0.8788 - val_loss: 0.2393 - val_acc: 0.6644 - val_competition_metric: 0.7803

Epoch 00031:

188/188 [==============================] - 74s 396ms/step - loss: 0.0926 - acc: 0.7300 - competition_metric: 0.8168 - val_loss: 0.2651 - val_acc: 0.6526 - val_competition_metric: 0.7237

Epoch 00012: val_competition_metric did not improve from 0.75310
Epoch 13/10000
188/188 [==============================] - 75s 397ms/step - loss: 0.0913 - acc: 0.7101 - competition_metric: 0.8204 - val_loss: 0.2519 - val_acc: 0.6905 - val_competition_metric: 0.6995

Epoch 00013: val_competition_metric did not improve from 0.75310
Epoch 14/10000
188/188 [==============================] - 74s 396ms/step - loss: 0.0774 - acc: 0.7096 - competition_metric: 0.8175 - val_loss: 0.2373 - val_acc: 0.6690 - val_competition_metric: 0.7503

Epoch 00014: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.

Epoch 00014: val_competition_metric did not improve from 0.75310
Epoch 15/10000
188/188 [==============================] - 74s 392ms/step - loss: 0.0630 - acc: 0.7106 - competition_metric: 0.8447 -

188/188 [==============================] - 75s 398ms/step - loss: 0.0352 - acc: 0.6692 - competition_metric: 0.8867 - val_loss: 0.2584 - val_acc: 0.6514 - val_competition_metric: 0.7769

Epoch 00042: val_competition_metric did not improve from 0.78070
Epoch 43/10000
188/188 [==============================] - 75s 397ms/step - loss: 0.0352 - acc: 0.6681 - competition_metric: 0.8914 - val_loss: 0.2585 - val_acc: 0.6506 - val_competition_metric: 0.7767

Epoch 00043: ReduceLROnPlateau reducing learning rate to 6.399999818995639e-09.

Epoch 00043: val_competition_metric did not improve from 0.78070
Epoch 00043: early stopping


In [27]:

epochs = 20000
BATCH_SIZE = 16

input_1 = 128
input_2 = 128
for fold_number in [0,1]:
    print ( 50*'-', ' FOLD ' + str(fold_number), 50*'-' )
    ids_train = df.ids_train_batch[df.fold != fold_number]
    ids_valid = df.ids_train_batch[df.fold == fold_number]

    model = Unet(backbone_name='resnet34', encoder_weights=None,input_shape=(256,256,3),freeze_encoder=False,
                            decoder_use_batchnorm = True,decoder_block_type = 'upsampling')
    model.compile(optimizers.Adam(lr = 0.0001), bce_dice_loss, ['accuracy',competition_metric])

    callbacks = [EarlyStopping(monitor='val_competition_metric',
                               patience=20,
                               verbose=1,
                               min_delta=1e-12,
                               mode='max'),
                 ReduceLROnPlateau(monitor='val_competition_metric',
                                   factor=0.2,
                                   patience=4,
                                   verbose=1,
                                   epsilon=1e-12,
                                   mode='max'),
                 ModelCheckpoint(monitor='val_competition_metric',
                                 filepath='4_origina_model_weights_fold_{}.hdf5'.format(fold_number),
                                 save_best_only=True,verbose = 1,
                                 mode='max')]
    print('number of train samples in fold ' +str(fold_number) + ' is ' + str(len(ids_train)))
    print('number of validation samples in fold ' +str(fold_number) + ' is ' + str(len(ids_valid)))
    
    model.load_weights('1_origina_model_weights_fold_{}.hdf5'.format(fold_number))
    #model.load_weights('AUTOENCODER.hdf5'.format(fold_number))
    model.fit_generator(generator=train_generator(ids_train),
                        steps_per_epoch=np.ceil(float(len(ids_train)) / float(BATCH_SIZE)),
                        epochs=10000,
                        verbose=1,
                        callbacks=callbacks,
                        validation_data=valid_generator(ids_valid),
                        validation_steps=np.ceil(float(len(ids_valid)) / float(BATCH_SIZE)))
    
    del model



--------------------------------------------------  FOLD 0 --------------------------------------------------


/home/n01z3/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


number of train samples in fold 0 is 3000
number of validation samples in fold 0 is 1000
Epoch 1/10000
188/188 [==============================] - 132s 705ms/step - loss: 0.3289 - acc: 0.7198 - competition_metric: 0.5200 - val_loss: 0.2788 - val_acc: 0.6975 - val_competition_metric: 0.6061

Epoch 00001: val_competition_metric improved from -inf to 0.60610, saving model to 4_origina_model_weights_fold_0.hdf5
Epoch 2/10000
188/188 [==============================] - 73s 390ms/step - loss: 0.2595 - acc: 0.7263 - competition_metric: 0.6313 - val_loss: 0.2347 - val_acc: 0.7145 - val_competition_metric: 0.6557

Epoch 00002: val_competition_metric improved from 0.60610 to 0.65570, saving model to 4_origina_model_weights_fold_0.hdf5
Epoch 3/10000
188/188 [==============================] - 75s 398ms/step - loss: 0.2067 - acc: 0.7271 - competition_metric: 0.6852 - val_loss: 0.2585 - val_acc: 0.7355 - val_competition_metric: 0.6297

Epoch 00003: val_competition_metric did not improve from 0.65570
E

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



188/188 [==============================] - 75s 398ms/step - loss: 0.0341 - acc: 0.7272 - competition_metric: 0.8934 - val_loss: 0.2507 - val_acc: 0.7041 - val_competition_metric: 0.7948

Epoch 00046: val_competition_metric did not improve from 0.79700
Epoch 47/10000
188/188 [==============================] - 75s 398ms/step - loss: 0.0350 - acc: 0.7257 - competition_metric: 0.8906 - val_loss: 0.2501 - val_acc: 0.7082 - val_competition_metric: 0.7931

Epoch 00047: ReduceLROnPlateau reducing learning rate to 3.199999980552093e-08.

Epoch 00047: val_competition_metric did not improve from 0.79700
Epoch 48/10000
188/188 [==============================] - 75s 399ms/step - loss: 0.0345 - acc: 0.7275 - competition_metric: 0.8929 - val_loss: 0.2498 - val_acc: 0.7051 - val_competition_metric: 0.7925

Epoch 00048: val_competition_metric did not improve from 0.79700
Epoch 49/10000
188/188 [==============================] - 75s 397ms/step - loss: 0.0345 - acc: 0.7263 - competition_metric: 0.8928 - 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Epoch 00032: val_competition_metric did not improve from 0.78870
Epoch 33/10000
188/188 [==============================] - 75s 398ms/step - loss: 0.0391 - acc: 0.6782 - competition_metric: 0.8835 - val_loss: 0.2364 - val_acc: 0.6501 - val_competition_metric: 0.7891

Epoch 00033: val_competition_metric improved from 0.78870 to 0.78910, saving model to 4_origina_model_weights_fold_1.hdf5
Epoch 34/10000
188/188 [==============================] - 75s 397ms/step - loss: 0.0393 - acc: 0.6784 - competition_metric: 0.8838 - val_loss: 0.2375 - val_acc: 0.6525 - val_competition_metric: 0.7856

Epoch 00034: val_competition_metric did not improve from 0.78910
Epoch 35/10000
188/188 [==============================] - 75s 397ms/step - loss: 0.0391 - acc: 0.6786 - competition_metric: 0.8855 - val_loss: 0.2385 - val_acc: 0.6514 - val_competition_metric: 0.7879

Epoch 00035: val_competition_metric did not improve from 0.78910
Epoch 36/10000
188/188 [==============================] - 75s 397ms/step - l

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
2+2

4